# Import libraries

In [1]:
import pandas as pd
import numpy as np
import copy, random, math
import ast
import folium

from geopy.distance import geodesic
from folium.plugins import MarkerCluster

# Loading data & Data preparation

In [2]:
distances = pd.read_csv('distances.csv')
establishments = pd.read_csv('establishments.csv')

# shift the column names to the left
distances.columns = pd.Index(list(distances.columns[1:]) + ['p_1000'])

very_small_establishments = establishments.iloc[0:21,:]
very_small_distances = distances.iloc[0:21,0:21]

small_establishments = establishments.iloc[0:101,:]
small_distances = distances.iloc[0:101,0:101]

medium_establishments = establishments.iloc[0:501,:]
medium_distances = distances.iloc[0:501,0:501]

large_establishments = establishments
large_distances = distances

# Problem definition

In [3]:
# convert the DataFrame to a dictionary
traveling_times = {}
for i in range(len(very_small_distances)):
    traveling_times[i] = very_small_distances.iloc[i].tolist()

num_establishments = len(traveling_times)-1
vehicles = int(num_establishments*0.1)
inspection_times = very_small_establishments[['Inspection Time']]
open_hours = very_small_establishments[['Opening Hours']]


def generate_random_solution():
    return np.random.randint(1, vehicles + 1, num_establishments)

def objective_function(chromosome, traveling_times, inspection_times, open_hours):
    total_travel_time = 0
    total_inspection_time = 0
    total_time_per_route = [0 for i in range(max(chromosome))]
    j = -1

    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]
    best_routes = []

    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)

    # Calculate the total travel time and inspection time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j += 1
        route_travel_time = 0
        route_inspection_time = 0
        current_time = 0
        # Add the departure/arrival establishment to the beginning and end of the route
        route = [0] + route + [0]
        
        # Use a greedy search algorithm to find the optimal path for the current route
        route = greedy_search(route, heuristic_distances)
        best_routes.append(route)
        
        start_time = ast.literal_eval(open_hours.iloc[route[1]][0]).index(1)  # Inspection start time of first establishment in route
        #print('start time: {}'.format(start_time))

        for i in range(1, len(route)):
            # Calculate traveling time between establishments
            current_establishment = route[i-1]
            next_establishment = route[i]
            travel_time = traveling_times[current_establishment][next_establishment]
            #route_travel_time += traveling_times[current_establishment][next_establishment]
            
            # Calculate inspection time and waiting time based on establishment's schedule
            current_inspection_time = inspection_times.iloc[next_establishment][0]*60 # Convert minutes to seconds
            current_open_hours = ast.literal_eval(open_hours.iloc[next_establishment][0])
            
            # Calculate waiting time if establishment is closed
            if i == 1:
                waiting_time = 0
            else:
                current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                #print('Current hour: {}'.format(current_hour))
                while current_open_hours[current_hour % 24] == 0:
                    current_hour += 1
                waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                #print('({} * 3600) - ({} + {} + ({}*3600))'.format(current_hour, current_time, travel_time, start_time))
                
            # Add inspection time and waiting time to current time
            if i > 1:
                current_time += max(waiting_time, 0)
            current_time += travel_time
            current_time += current_inspection_time
            
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(i, current_establishment, next_establishment, current_inspection_time, current_open_hours, travel_time, waiting_time, current_time))
            # Reset waiting time
            waiting_time = 0
            
            # Add inspection time and traveling time to route times
            #route_inspection_time += current_inspection_time
            route_travel_time = 0

        # Add total time for current route to total time per route
        total_time_per_route[j] = current_time
        #print('total time per route ----> {}'.format(total_time_per_route))

    # Add total time for all routes to total travel time
    total_travel_time += sum(total_time_per_route)
    
    return -total_travel_time, best_routes # Minimize the sum of inspection and travel time

def objective_function_a_star(chromosome, traveling_times, inspection_times, open_hours):
    
    total_travel_time = 0
    l = 1
    
    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]
    best_routes = []
    
    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)
    
    # Calculate the total travel time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j = 1
        current_time = 0
        
        # Create a set with all establishments in the route
        route_establishments = set(route)
        route_len = len(route_establishments)
        
        # Add the depot to the beginning of the route
        route = [0]
        
        # Initialize the route travel time as zero
        route_travel_time = 0
        
        while route_establishments:
            # Calculate the A* algorithm for the next establishment
            current_establishment = route[-1]
            next_establishment = None
            best_score = float('inf')
            for e in route_establishments:
                i = 1
                # Calculate the total time (travel time, inspection time, waiting time, heuristic value) for the next establishment
                travel_time = traveling_times[route[-1]][e]
                inspection_time = inspection_times.iloc[e][0] * 60
                current_open_hours = ast.literal_eval(open_hours.iloc[e][0])
                if len(route_establishments) == route_len:
                    start_time = current_open_hours.index(1)
                i += 1
                # Calculate waiting time if establishment is closed
                if len(route) == 1:
                    waiting_time = 0
                else:
                    current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                    while current_open_hours[current_hour % 24] == 0:
                        current_hour += 1
                    waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                
                # Add inspection time, waiting time, and travel time to the current time
                time_to_next = max(waiting_time, 0) + inspection_time + travel_time
                total_time = current_time + time_to_next
                heuristic_value = heuristic_distances[e][0]
                score = total_time + heuristic_value
                
                # If the current establishment has the smallest score, update next_establishment
                if score < best_score:
                    '''
                    best_waiting_time = max(waiting_time, 0)
                    best_open_hours = current_open_hours
                    best_inspection_time = inspection_time
                    if len(route) == 1:
                        best_start_time = start_time
                    else:
                        best_current_hour = current_hour
                    best_current_time = total_time
                    best_travel_time = travel_time
                    '''
                    time_to_next_establishment = time_to_next
                    next_establishment = e
                    best_score = score
            '''
            if len(route) > 1:
                print('Current hour: {}'.format(best_current_hour))
                print('({} * 3600) - ({} + {} + ({}*3600))'.format(best_current_hour, current_time, best_travel_time, best_start_time))
            else:
                print('start time: {}'.format(best_start_time))
            '''
            
            # Add next establishment to route and remove from set
            route_establishments.remove(next_establishment)
            route.append(next_establishment)
            
            #print('Update current time: {} + {}'.format(current_time, time_to_next_establishment))
            # Update the current time and route travel time
            current_time += time_to_next_establishment
            #route_travel_time += travel_time
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, next inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(j, current_establishment, next_establishment, best_inspection_time, best_open_hours, best_travel_time, best_waiting_time, best_current_time))
        
            j += 1
        # Add the depot to the end of the route
        route.append(0)
        best_routes.append(route)
        #print('Route: {}'.format(route))
        # Calculate the total time for the current route and add to total time per route
        total_time_per_route = current_time + traveling_times[route[-2]][0]
        #print('Traveling time back to the depot: {}'.format(traveling_times[route[-2]][0]))
        #print('Total travel time in route {}: {}'.format(l, total_time_per_route))
        total_travel_time += total_time_per_route
        l += 1
        
    return -total_travel_time, best_routes # Minimize the total travel time

def geodesic_time_matrix(establishments, speed_kmph):
    """
    Calculates the travel time matrix between all combinations of establishments based on their
    latitude and longitude coordinates, assuming a given speed in km/h.
    
    Parameters:
    establishments (pandas.DataFrame): A DataFrame containing the latitude and longitude coordinates of all establishments.
    speed_kmph (float): The speed of travel in km/h.
    
    Returns:
    pandas.DataFrame: A DataFrame containing the travel time matrix between all combinations of establishments in seconds.
    """
    # Create a 2D array of establishment coordinates
    coordinates = establishments[['Latitude', 'Longitude']].values
    
    # Initialize an empty time matrix with the same shape as the coordinates matrix
    time_matrix = pd.DataFrame(index=establishments.index, columns=establishments.index)
    
    # Calculate pairwise travel times between all establishments
    for i in range(len(coordinates)):
        for j in range(len(coordinates)):
            # Calculate the distance between establishments i and j using geodesic distance
            distance_km = geodesic(coordinates[i], coordinates[j]).km
            
            # Calculate the travel time in seconds using the speed in km/h
            travel_time_sec = (distance_km / speed_kmph) * 3600
            
            # Set the travel time in the time matrix
            time_matrix.iloc[i, j] = travel_time_sec
    
    return time_matrix

def greedy_search(route, heuristic):
    current_node = 0
    visited = [False] * (max(route) + 1)
    visited[current_node] = True
    path = [current_node]
    while len(path) < len(route):
        next_node = -1
        min_distance = float('inf')
        for i in range(len(route)):
            if not visited[route[i]]:
                distance = heuristic[route[i]][route[0]]
                if distance < min_distance:
                    next_node = route[i]
                    min_distance = distance
        visited[next_node] = True
        path.append(next_node)
        current_node = next_node
    path.pop() # remove last element (should be 0)
    path.append(0)
    return path

def display_routes(routes):
    # Define the map center and zoom level
    center = [41.160304, -8.602478]
    zoom = 10.4

    # Create a map object
    tile = 'OpenStreetMap'
    map = folium.Map(location=center, zoom_start=zoom, tiles=tile)

    # Define the color palette for the routes
    route_colors = ['#d62728','#3388ff','#33cc33','#ff9933','#800080','#ff3399','#808080','#f5f5dc','#32174d','#ffffff',
              '#5f9ea0','#d3d3d3','#add8e6','#00008b','#90ee90','#006400','#8b0000','#ff6666']
    marker_colors = ['red', 'blue', 'green', 'orange', 'purple', 'pink', 'gray', 'beige', 'darkpurple', 'white', 
                     'cadetblue', 'lightgray', 'lightblue', 'darkblue', 'lightgreen', 'darkgreen', 'darkred', 'lightred']

    # Create checkboxes for each route
    checkboxes = []
    for i, route in enumerate(routes):
        label = f"Route {i+1}"
        checkbox = folium.FeatureGroup(name=label, overlay=True, control=True)
        for k, j in enumerate(route[1:-1]):
            folium.Marker(
                location=(establishments.iloc[j]['Latitude'], establishments.iloc[j]['Longitude']),
                icon=folium.Icon(color=marker_colors[i % len(route_colors)]),
                tooltip=f"Visit order: {k+1}",
                popup=f"{establishments.iloc[j]['Address']}",
            ).add_to(checkbox)
        folium.PolyLine(
            locations=[(establishments.iloc[j]['Latitude'], establishments.iloc[j]['Longitude']) for j in route],
            color=route_colors[i % len(route_colors)],
            popup=label,
        ).add_to(checkbox)
        checkboxes.append(checkbox)
    
    # Add caution marker to the depot
    folium.Marker(location=center, icon=folium.Icon(icon='home', prefix='fa', color='black'), tooltip='Depot', 
                 popup=f"{establishments.iloc[0]['Address']}").add_to(map)
    
    # Add the checkboxes to the map
    for checkbox in checkboxes:
        checkbox.add_to(map)
    
    # Add a layer control to the map
    folium.LayerControl().add_to(map)
    
    # Display the map
    return map

In [4]:
heuristic_distances = geodesic_time_matrix(establishments, 70)

# Subtract the heuristic distance matrix from the travel time matrix
difference_matrix = heuristic_distances - distances

# Check if there are any positive values in the difference matrix (checking for an optimistic heuristics)
if (difference_matrix > 0).any().any():
    print('There is at least one travel time that is lower than the heuristic value!')
else:
    print('All travel times are greater than or equal to the heuristic value.')

All travel times are greater than or equal to the heuristic value.


# Genetic Algorithm

In [5]:
def midpoint_crossover(solution_1, solution_2):
    mid_index = math.trunc(len(solution_1) / 2)
    child_1 = np.append(solution_1[0:mid_index], solution_2[mid_index:]) 
    child_2 = np.append(solution_2[0:mid_index], solution_1[mid_index:])
    return child_1, child_2

def randompoint_crossover(solution_1, solution_2):
    length = len(solution_1)
    crossover_point = random.randint(1, length - 1)
    child_1 = np.concatenate([solution_1[:crossover_point], solution_2[crossover_point:]])
    child_2 = np.concatenate([solution_2[:crossover_point], solution_1[crossover_point:]])
    return child_1, child_2

def uniform_crossover(solution_1, solution_2):
    """
    Performs uniform crossover (UX) on two solutions.
    
    Args:
        solution_1 (list): The first solution.
        solution_2 (list): The second solution.
        
    Returns:
        Two new solutions created by randomly swapping genes between solution_1 and solution_2.
    """
    # Initialize children as copies of the parent solutions
    child_1 = solution_1.copy()
    child_2 = solution_2.copy()

    # Determine the genes to swap
    genes_to_swap = np.random.choice(len(solution_1), len(solution_1), replace=False)

    # Swap the genes between the two children
    for i in genes_to_swap:
        if child_1[i] != child_2[i]:
            child_1[i], child_2[i] = child_2[i], child_1[i]

    return child_1, child_2

def mutate_solution_1(solution):
    index_1 = np.random.randint(0, len(solution))
    index_2 = (index_1 + np.random.randint(0, len(solution))) % (len(solution) - 1) # Efficient way to generate a non-repeated index
    solution[index_1], solution[index_2] = solution[index_2], solution[index_1]
    return solution

def mutate_solution_2(solution):
    index = np.random.randint(0, len(solution))
    solution[index] = np.random.randint(1, vehicles + 1)
    return solution

def generate_population(population_size):
    solutions = []
    for i in range(population_size):
        solutions.append(generate_random_solution())
    return solutions

def print_population(population, objective_func):
    solutions = []
    for i in range(len(population)):
        print(f"Solution {i+1}: {population[i]}, {-objective_func(population[i], traveling_times, inspection_times, open_hours)[0]}")
        
def tournament_select(population, tournament_size, objective_func):
    population_copy = copy.deepcopy(population)
    best_solution = population_copy[0]
    best_score = objective_func(population_copy[0], traveling_times, inspection_times, open_hours)[0]
    for i in range(tournament_size):
        index = np.random.randint(0, len(population_copy))
        score = objective_func(population_copy[index], traveling_times, inspection_times, open_hours)[0]
        if score > best_score:
            best_score = score
            best_solution = population_copy[index]
        del population_copy[index]
    return best_solution

def get_greatest_fit(population, objective_func):
    best_solution = population[0]
    best_score, best_routes = objective_func(population[0], traveling_times, inspection_times, open_hours)
    for i in range(1, len(population)):
        score, routes = objective_func(population[i], traveling_times, inspection_times, open_hours)
        if score > best_score:
            best_score = score
            best_solution = population[i]
            best_routes = routes
    return best_solution, best_score, best_routes

def replace_least_fittest(population, offspring, objective_func):
    least_fittest_index = 0
    least_fittest_value = objective_func(population[0], traveling_times, inspection_times, open_hours)[0]
    for i in range(1, len(population)):
        score = objective_func(population[i], traveling_times, inspection_times, open_hours)[0]
        if score < least_fittest_value:
            least_fittest_value = score
            least_fittest_index = i
    population[least_fittest_index] = offspring

def roulette_select(population, objective_func):
    score_sum = sum([objective_func(solution, traveling_times, inspection_times, open_hours)[0] for solution in population])
    selection_probabilities = [objective_func(solution, traveling_times, inspection_times, open_hours)[0] / score_sum for solution in population]
    return population[np.random.choice(len(population), p=selection_probabilities)]

In [6]:
#Test Crossover
s1 = generate_random_solution()
s2 = generate_random_solution()
print('solution 1 -----> {}'.format(s1))
print('solution 2 -----> {}'.format(s2))
c1, c2 = midpoint_crossover(s1, s2)
print('child 1 -----> {}'.format(c1))
print('child 2 -----> {}'.format(c2))
c1, c2 = randompoint_crossover(s1, s2)
print('new child 1 -----> {}'.format(c1))
print('new child 2 -----> {}'.format(c2))
c1, c2 = uniform_crossover(s1, s2)
print('new child 1 ux -----> {}'.format(c1))
print('new child 2 ux -----> {}'.format(c2))
#Test Mutation
c3 = mutate_solution_1(c1)
c4 = mutate_solution_1(c2)
print('mutation of child 1 -----> {}'.format(c3))
print('mutation of child 2 -----> {}'.format(c4))

solution 1 -----> [1 1 1 2 2 1 1 2 1 2 2 1 2 1 2 1 2 1 2 2]
solution 2 -----> [1 2 1 1 1 1 1 2 2 1 2 2 2 2 2 1 2 1 1 1]
child 1 -----> [1 1 1 2 2 1 1 2 1 2 2 2 2 2 2 1 2 1 1 1]
child 2 -----> [1 2 1 1 1 1 1 2 2 1 2 1 2 1 2 1 2 1 2 2]
new child 1 -----> [1 2 1 1 1 1 1 2 2 1 2 2 2 2 2 1 2 1 1 1]
new child 2 -----> [1 1 1 2 2 1 1 2 1 2 2 1 2 1 2 1 2 1 2 2]
new child 1 ux -----> [1 2 1 1 1 1 1 2 2 1 2 2 2 2 2 1 2 1 1 1]
new child 2 ux -----> [1 1 1 2 2 1 1 2 1 2 2 1 2 1 2 1 2 1 2 2]
mutation of child 1 -----> [1 2 1 1 1 1 1 2 2 1 2 2 2 2 2 1 2 1 1 1]
mutation of child 2 -----> [1 1 1 2 2 1 1 2 1 2 2 1 2 1 2 1 2 1 2 2]


In [7]:
pop = generate_population(10)
print_population(pop, objective_function_a_star)

tournament = tournament_select(pop, 2, objective_function_a_star)
print('Tournament -----> {}'.format(tournament))

best_fit = get_greatest_fit(pop, objective_function_a_star)
print('Greatest fit -----> {}'.format(best_fit))

roulette = roulette_select(pop, objective_function_a_star)
print('Roulette -----> {}'.format(roulette))

Solution 1: [2 2 2 2 2 1 1 2 2 2 1 1 1 2 2 2 1 1 1 1], 144261.90000000002
Solution 2: [1 1 2 1 2 1 1 2 2 1 1 2 1 1 1 1 2 1 1 2], 159657.9
Solution 3: [1 2 1 2 2 1 1 1 2 1 2 1 2 1 1 1 2 1 2 1], 140176.90000000002
Solution 4: [1 1 2 1 1 2 2 1 1 1 1 1 2 2 2 1 1 1 2 1], 144866.2
Solution 5: [2 1 1 1 2 2 1 1 2 2 1 1 2 2 1 1 2 2 1 1], 144750.0
Solution 6: [2 2 2 1 2 1 1 2 1 2 2 1 2 1 1 2 1 1 2 1], 159657.9
Solution 7: [2 2 1 1 1 2 1 1 2 2 2 2 2 1 2 2 2 1 2 1], 156260.1
Solution 8: [1 2 2 2 2 1 2 1 1 2 1 1 1 1 1 2 2 1 1 1], 152725.1
Solution 9: [1 2 2 2 2 1 1 1 2 2 2 2 2 2 1 2 1 2 2 1], 144440.0
Solution 10: [1 1 2 2 2 2 2 1 2 2 1 1 2 2 1 2 2 1 1 2], 139417.0
Tournament -----> [1 1 2 2 2 2 2 1 2 2 1 1 2 2 1 2 2 1 1 2]
Greatest fit -----> (array([1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2]), -139417.0, [[0, 19, 2, 1, 15, 11, 12, 8, 18, 0], [0, 17, 5, 6, 16, 13, 4, 14, 20, 7, 3, 9, 10, 0]])
Roulette -----> [1 1 2 1 2 1 1 2 2 1 1 2 1 1 1 1 2 1 1 2]


In [8]:
def genetic_algorithm(num_iterations, population_size, crossover_func, mutation_func, objective_func, log=False):
    population = generate_population(population_size)
    
    best_solution = population[0] # Initial solution
    best_score = objective_func(population[0], traveling_times, inspection_times, open_hours)[0]
    best_solution_generation = 0 # Generation on which the best solution was found
    
    generation_no = 0
    
    print(f"Initial solution: {best_solution}, score: {-best_score}")
    
    while(num_iterations > 0):
        
        generation_no += 1
        
        tournment_winner_sol = tournament_select(population, 4, objective_func)
        roulette_winner_sol = roulette_select(population, objective_func)
        
        # Next generation
        crossover_sol_1, crossover_sol_2 = crossover_func(tournment_winner_sol, roulette_winner_sol)
    
        if np.random.randint(0, 10) < 1: # 40% chance to perform mutation
            replace_least_fittest(population, mutation_func(crossover_sol_1), objective_func)
            replace_least_fittest(population, mutation_func(crossover_sol_2), objective_func)
        else:
            replace_least_fittest(population, crossover_sol_1, objective_func)
            replace_least_fittest(population, crossover_sol_2, objective_func)
        
        # Checking the greatest fit among the current population
        greatest_fit, greatest_fit_score, best_routes = get_greatest_fit(population, objective_func)
        if greatest_fit_score > best_score:
            best_solution = greatest_fit
            best_score = greatest_fit_score
            best_routes
            best_solution_generation = generation_no
            if log:
                print(f"\nGeneration: {generation_no }")
                print(f"Solution: score: {-best_score}")
                print_population(population, objective_func)
        else:
            num_iterations -= 1
        
    print(f"  Final solution: {best_solution}, score: {-best_score}, best routes: {best_routes}")
    print(f"  Found on generation {best_solution_generation}")
    
    return best_solution, best_routes

In [9]:
print("\nGeneticAlgorithm:\n")
solution, routes = genetic_algorithm(500, 30, randompoint_crossover, mutate_solution_1, objective_function, True)


GeneticAlgorithm:

Initial solution: [1 1 2 2 2 2 1 1 1 1 2 1 2 2 1 2 2 1 1 1], score: 242115.3

Generation: 1
Solution: score: 163414.5
Solution 1: [1 1 2 2 2 2 1 1 1 1 2 1 2 2 1 2 2 1 1 1], 242115.3
Solution 2: [1 2 1 1 2 1 2 2 2 2 2 2 1 1 2 1 1 2 1 2], 213233.7
Solution 3: [2 1 2 2 1 1 2 2 2 2 1 2 1 1 1 1 1 1 2 1], 244730.6
Solution 4: [2 1 1 1 2 1 1 2 1 2 1 1 1 1 2 2 2 2 1 2], 248375.8
Solution 5: [1 1 1 1 1 1 1 1 2 1 2 1 2 1 1 1 2 2 2 2], 172269.2
Solution 6: [2 1 2 2 2 2 2 2 1 2 2 1 2 2 1 2 2 2 2 2], 164527.90000000002
Solution 7: [1 2 2 1 2 2 2 1 2 1 2 1 2 2 1 2 1 1 1 2], 217325.9
Solution 8: [2 1 1 2 2 2 1 2 2 1 1 1 1 2 2 2 1 1 2 2], 242457.90000000002
Solution 9: [2 1 2 1 1 1 2 1 1 1 2 1 1 2 2 1 2 2 1 1], 177347.8
Solution 10: [1 1 1 1 2 1 2 2 1 2 1 2 1 2 2 2 1 1 2 1], 246125.9
Solution 11: [1 2 2 1 1 2 1 1 2 1 2 1 2 2 2 1 2 2 2 2], 215474.1
Solution 12: [1 2 2 1 2 1 1 1 2 2 2 1 1 1 1 2 1 1 2 2], 190741.90000000002
Solution 13: [2 1 1 1 2 1 1 2 1 1 2 1 2 1 2 2 1 1 1 2], 23491

In [10]:
score, routes = objective_function(solution, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 18, 8, 2, 4, 11, 10, 9, 0], [0, 19, 17, 5, 12, 6, 16, 1, 3, 13, 14, 7, 20, 15, 0]]
Objective function's values ---> 152926.5


In [11]:
print("\nGeneticAlgorithm:\n")
solution, routes = genetic_algorithm(100, 30, uniform_crossover, mutate_solution_1, objective_function_a_star, True)


GeneticAlgorithm:

Initial solution: [2 1 2 2 1 2 1 2 2 2 2 2 2 2 1 1 2 2 2 2], score: 129551.90000000001

Generation: 1
Solution: score: 110889.4
Solution 1: [2 1 2 2 1 2 1 2 2 2 2 2 2 2 1 1 2 2 2 2], 129551.90000000001
Solution 2: [1 2 1 2 1 2 1 2 1 2 2 2 1 1 2 1 2 2 2 2], 169205.7
Solution 3: [1 2 2 2 1 2 2 2 1 1 2 1 2 1 1 2 2 1 2 2], 111910.6
Solution 4: [1 1 2 2 2 2 1 2 1 2 2 1 1 1 2 2 1 1 1 1], 141447.2
Solution 5: [1 2 2 1 1 1 1 2 1 2 2 2 1 2 2 1 2 1 2 2], 161705.0
Solution 6: [2 1 1 1 2 2 1 2 1 2 1 2 1 2 2 1 2 1 2 2], 162727.80000000002
Solution 7: [2 1 1 1 1 2 1 2 1 2 2 1 2 2 1 1 1 1 1 1], 134603.1
Solution 8: [1 2 2 2 1 2 1 2 2 2 1 1 2 2 2 1 1 1 2 2], 142967.8
Solution 9: [2 1 1 2 2 1 1 2 2 1 2 1 2 2 2 2 1 2 2 2], 162262.2
Solution 10: [2 2 2 2 1 1 2 2 2 1 1 1 1 1 2 2 2 1 2 2], 118398.29999999999
Solution 11: [1 1 1 1 2 1 2 2 1 2 1 2 2 1 1 1 2 1 1 1], 146657.3
Solution 12: [1 1 1 2 1 2 2 1 1 1 1 2 2 1 1 1 1 2 2 2], 111240.8
Solution 13: [2 1 1 1 1 2 2 2 1 1 2 1 2 2 1 2 2 1 2

In [12]:
score, routes = objective_function_a_star(solution, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 5, 2, 18, 3, 4, 15, 10, 12, 9, 0], [0, 19, 17, 16, 6, 14, 11, 1, 13, 20, 7, 8, 0]]
Objective function's values ---> 110889.4


# Simulated Annealing

In [9]:
# Neighborhood definition

def get_neighbor_solution1(solution):
    neighbor = copy.deepcopy(solution)
    establishment_to_mutate = np.random.randint(0, num_establishments)
    neighbor[establishment_to_mutate] = (neighbor[establishment_to_mutate] + np.random.randint(1, vehicles)-1) % vehicles + 1
    return neighbor

# Exchange the vehicles of two establishments
def get_neighbor_solution2(solution):
    neighbor_solution = copy.deepcopy(solution)
    establishment1 = np.random.randint(0, num_establishments)
    establishment2 = (establishment1 + np.random.randint(1, num_establishments)) % num_establishments
    neighbor_solution[establishment1], neighbor_solution[establishment2] = neighbor_solution[establishment2], neighbor_solution[establishment1]
    return neighbor_solution

# Neighbour 1 or 2 with 50% each
def get_neighbor_solution3(solution):
    if (np.random.randint(0,2)==0):
        return get_neighbor_solution1(solution)
    else:
        return get_neighbor_solution2(solution)

In [10]:
#Test Neighbours
s = generate_random_solution()
print(s)
for d1 in range(0,20):
    print('Neighbor number {} with get_neighbor_solution1 ---> {}'.format(d1+1, get_neighbor_solution1(s)))

for d1 in range(0,20):
    print('Neighbor number {} with get_neighbor_solution2 ---> {}'.format(d1+1, get_neighbor_solution2(s)))

[2 2 1 1 2 2 1 2 1 1 1 2 1 2 1 1 1 1 1 1]
Neighbor number 1 with get_neighbor_solution1 ---> [2 2 1 1 2 2 1 2 1 1 1 2 1 2 2 1 1 1 1 1]
Neighbor number 2 with get_neighbor_solution1 ---> [2 2 1 1 2 2 2 2 1 1 1 2 1 2 1 1 1 1 1 1]
Neighbor number 3 with get_neighbor_solution1 ---> [2 2 1 1 2 2 1 2 2 1 1 2 1 2 1 1 1 1 1 1]
Neighbor number 4 with get_neighbor_solution1 ---> [2 2 1 1 2 2 1 2 1 2 1 2 1 2 1 1 1 1 1 1]
Neighbor number 5 with get_neighbor_solution1 ---> [2 2 1 2 2 2 1 2 1 1 1 2 1 2 1 1 1 1 1 1]
Neighbor number 6 with get_neighbor_solution1 ---> [2 2 1 1 2 2 1 2 1 1 1 2 1 2 1 1 1 1 1 2]
Neighbor number 7 with get_neighbor_solution1 ---> [2 2 1 1 2 2 1 2 1 1 1 2 2 2 1 1 1 1 1 1]
Neighbor number 8 with get_neighbor_solution1 ---> [2 2 1 1 2 1 1 2 1 1 1 2 1 2 1 1 1 1 1 1]
Neighbor number 9 with get_neighbor_solution1 ---> [2 2 1 1 2 2 1 2 1 1 2 2 1 2 1 1 1 1 1 1]
Neighbor number 10 with get_neighbor_solution1 ---> [2 2 1 1 2 2 1 2 1 1 1 2 1 2 1 1 1 2 1 1]
Neighbor number 11 with get

In [11]:
def get_sa_solution(num_iterations, neighbor_operator, traveling_times, inspection_times, open_hours, objective_func,log=False):
    iteration = 0;
    temperature = 1000;
    solution = generate_random_solution() # Best solution after 'num_iterations' iterations without improvement
    score, routes = objective_func(solution, traveling_times, inspection_times, open_hours)
    
    best_solution = copy.deepcopy(solution)
    best_score = score
    best_routes = routes
    
    print(f"Init Solution:  {best_solution}, score: {-best_score}")
    
    while iteration < num_iterations:
        temperature = temperature * 0.999  # Test with different cooling schedules
        iteration += 1
        neighbor_solution = neighbor_operator(best_solution)  #Test with Neighbour 1, 2 and 3
        neighbor_score, neighbor_routes = objective_func(neighbor_solution, traveling_times, inspection_times, open_hours)
        
        delta = neighbor_score - score 
        
        if delta > 0 or np.exp(-delta/temperature) > random.random():
            solution = neighbor_solution
            score = neighbor_score
            routes = neighbor_routes
            if score > best_score:
                iteration = 0
                best_solution = copy.deepcopy(solution)
                best_score = score
                best_routes = routes
                if log:
                    print(f"Solution: score: {-best_score},  Temp: {temperature}")
                
    print(f"Final Solution: {best_solution}, score: {-best_score}")
    return best_solution, best_routes

In [12]:
print("\nSimulated Annealing:\n")
solution, routes = get_sa_solution(2000, get_neighbor_solution1, traveling_times, inspection_times, open_hours, objective_function, True)


Simulated Annealing:

Init Solution:  [1 2 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 2 1 2], score: 179513.7
Solution: score: 176061.70000000004,  Temp: 999.0
Solution: score: 168447.2,  Temp: 997.0029989999999
Solution: score: 161971.60000000003,  Temp: 987.0777147137147
Solution: score: 159051.10000000003,  Temp: 977.2512378214516
Solution: score: 152133.40000000002,  Temp: 967.5225846837673
Solution: score: 151366.50000000003,  Temp: 965.5885070369843
Final Solution: [1 1 1 1 1 1 2 1 1 1 1 2 2 1 1 1 1 1 1 2], score: 151366.50000000003


In [13]:
score, routes = objective_function(solution, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 17, 5, 18, 8, 6, 16, 2, 1, 3, 14, 4, 11, 10, 15, 9, 0], [0, 12, 13, 7, 20, 0]]
Objective function's values ---> 151366.50000000003


In [14]:
print("\nSimulated Annealing:\n")
solution, routes = get_sa_solution(2000, get_neighbor_solution1, traveling_times, inspection_times, open_hours, objective_function_a_star, True)


Simulated Annealing:

Init Solution:  [1 2 1 2 1 1 1 1 1 1 2 1 1 2 1 2 2 2 2 1], score: 130001.6
Solution: score: 121984.00000000001,  Temp: 983.1353223738244
Solution: score: 119930.19999999998,  Temp: 978.2294672887404
Solution: score: 116165.3,  Temp: 972.3747443770956
Final Solution: [1 2 1 2 1 1 1 2 1 2 2 1 1 2 1 2 2 1 2 1], score: 116165.3


In [15]:
score, routes = objective_function_a_star(solution, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 5, 6, 1, 13, 15, 20, 7, 12, 18, 3, 9, 0], [0, 19, 17, 2, 16, 4, 11, 14, 8, 10, 0]]
Objective function's values ---> 116165.3


# Tabu Search

In [16]:
def get_tabu_search_solution(num_iterations, tabu_list_length, neighbor_operator, traveling_times, inspection_times, open_hours, objective_func,log=False):
    current_solution = generate_random_solution()
    current_score, current_route = objective_func(current_solution, traveling_times, inspection_times, open_hours)
    
    best_solution = current_solution
    best_score = current_score
    best_route = current_route
    
    tabu_list = []
    
    for iteration in range(num_iterations):
        neighbor_solutions = []
        neighbor_scores = []
        neighbor_routes = []
        tabu_neighbors = []
        
        for i in range(10):
            neighbor_solution = neighbor_operator(current_solution)
            neighbor_score, neighbor_route = objective_func(neighbor_solution, traveling_times, inspection_times, open_hours)
            if neighbor_solution.tolist() not in tabu_list:
                neighbor_solutions.append(neighbor_solution)
                neighbor_scores.append(neighbor_score)
                neighbor_routes.append(neighbor_route)
            else:
                tabu_neighbors.append(neighbor_solution)
        
        if len(neighbor_scores) > 0:
            best_neighbor_index = np.argmax(neighbor_scores)
            best_neighbor_score = neighbor_scores[best_neighbor_index]
            best_neighbor_solution = neighbor_solutions[best_neighbor_index]
            best_neighbor_route = neighbor_routes[best_neighbor_index]
            
            if best_neighbor_score > best_score:
                best_solution = best_neighbor_solution
                best_score = best_neighbor_score
                best_route = best_neighbor_route
            
            current_solution = best_neighbor_solution
            current_score = best_neighbor_score
            current_route = best_neighbor_route
            
            tabu_list.append(current_solution.tolist())
            if len(tabu_list) > tabu_list_length:
                tabu_list.pop(0)
                
        elif len(tabu_neighbors) > 0:
            best_tabu_index = np.argmax([objective_func(x, traveling_times, inspection_times, open_hours)[0] for x in tabu_neighbors])
            best_tabu_solution = tabu_neighbors[best_tabu_index]
            
            current_solution = best_tabu_solution
            current_score, current_route = objective_func(current_solution, traveling_times, inspection_times, open_hours)
            
            tabu_list.append(current_solution.tolist())
            if len(tabu_list) > tabu_list_length:
                tabu_list.pop(0)
        
        if log:
            print(f"Iteration {iteration}, Best Score: {-best_score}")
    
    print(f"Best solution: {best_solution}, Best Score: {-best_score}, Routes: {best_route}")
    
    return best_solution, best_route

In [17]:
print("\nTabu Search:\n")
solution, routes = get_tabu_search_solution(5000, 500, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function, True)


Tabu Search:

Iteration 0, Best Score: 197182.30000000005
Iteration 1, Best Score: 189783.7
Iteration 2, Best Score: 164464.2
Iteration 3, Best Score: 164464.2
Iteration 4, Best Score: 164464.2
Iteration 5, Best Score: 164464.2
Iteration 6, Best Score: 159070.5
Iteration 7, Best Score: 157901.80000000002
Iteration 8, Best Score: 157797.19999999998
Iteration 9, Best Score: 155286.1
Iteration 10, Best Score: 155286.1
Iteration 11, Best Score: 155286.1
Iteration 12, Best Score: 155286.1
Iteration 13, Best Score: 153877.59999999998
Iteration 14, Best Score: 153877.59999999998
Iteration 15, Best Score: 153877.59999999998
Iteration 16, Best Score: 153877.59999999998
Iteration 17, Best Score: 153877.59999999998
Iteration 18, Best Score: 153877.59999999998
Iteration 19, Best Score: 153877.59999999998
Iteration 20, Best Score: 153877.59999999998
Iteration 21, Best Score: 153877.59999999998
Iteration 22, Best Score: 153855.0
Iteration 23, Best Score: 153855.0
Iteration 24, Best Score: 153855.0


Iteration 228, Best Score: 128726.6
Iteration 229, Best Score: 128726.6
Iteration 230, Best Score: 128726.6
Iteration 231, Best Score: 128726.6
Iteration 232, Best Score: 128726.6
Iteration 233, Best Score: 128726.6
Iteration 234, Best Score: 128726.6
Iteration 235, Best Score: 128726.6
Iteration 236, Best Score: 128726.6
Iteration 237, Best Score: 128726.6
Iteration 238, Best Score: 128726.6
Iteration 239, Best Score: 128726.6
Iteration 240, Best Score: 128726.6
Iteration 241, Best Score: 128726.6
Iteration 242, Best Score: 128726.6
Iteration 243, Best Score: 128726.6
Iteration 244, Best Score: 128726.6
Iteration 245, Best Score: 128726.6
Iteration 246, Best Score: 128726.6
Iteration 247, Best Score: 128726.6
Iteration 248, Best Score: 128726.6
Iteration 249, Best Score: 128726.6
Iteration 250, Best Score: 128726.6
Iteration 251, Best Score: 128726.6
Iteration 252, Best Score: 128726.6
Iteration 253, Best Score: 128726.6
Iteration 254, Best Score: 128726.6
Iteration 255, Best Score: 1

Iteration 457, Best Score: 128726.6
Iteration 458, Best Score: 128726.6
Iteration 459, Best Score: 128726.6
Iteration 460, Best Score: 128726.6
Iteration 461, Best Score: 128726.6
Iteration 462, Best Score: 128726.6
Iteration 463, Best Score: 128726.6
Iteration 464, Best Score: 128726.6
Iteration 465, Best Score: 128726.6
Iteration 466, Best Score: 128726.6
Iteration 467, Best Score: 128726.6
Iteration 468, Best Score: 128726.6
Iteration 469, Best Score: 128726.6
Iteration 470, Best Score: 128726.6
Iteration 471, Best Score: 128726.6
Iteration 472, Best Score: 128726.6
Iteration 473, Best Score: 128726.6
Iteration 474, Best Score: 128726.6
Iteration 475, Best Score: 128726.6
Iteration 476, Best Score: 128726.6
Iteration 477, Best Score: 128726.6
Iteration 478, Best Score: 128726.6
Iteration 479, Best Score: 128726.6
Iteration 480, Best Score: 128726.6
Iteration 481, Best Score: 128726.6
Iteration 482, Best Score: 128726.6
Iteration 483, Best Score: 128726.6
Iteration 484, Best Score: 1

Iteration 679, Best Score: 123019.20000000001
Iteration 680, Best Score: 123019.20000000001
Iteration 681, Best Score: 123019.20000000001
Iteration 682, Best Score: 123019.20000000001
Iteration 683, Best Score: 123019.20000000001
Iteration 684, Best Score: 123019.20000000001
Iteration 685, Best Score: 123019.20000000001
Iteration 686, Best Score: 123019.20000000001
Iteration 687, Best Score: 123019.20000000001
Iteration 688, Best Score: 123019.20000000001
Iteration 689, Best Score: 123019.20000000001
Iteration 690, Best Score: 123019.20000000001
Iteration 691, Best Score: 123019.20000000001
Iteration 692, Best Score: 123019.20000000001
Iteration 693, Best Score: 123019.20000000001
Iteration 694, Best Score: 123019.20000000001
Iteration 695, Best Score: 123019.20000000001
Iteration 696, Best Score: 123019.20000000001
Iteration 697, Best Score: 123019.20000000001
Iteration 698, Best Score: 123019.20000000001
Iteration 699, Best Score: 123019.20000000001
Iteration 700, Best Score: 123019.

Iteration 859, Best Score: 123019.20000000001
Iteration 860, Best Score: 123019.20000000001
Iteration 861, Best Score: 123019.20000000001
Iteration 862, Best Score: 123019.20000000001
Iteration 863, Best Score: 123019.20000000001
Iteration 864, Best Score: 123019.20000000001
Iteration 865, Best Score: 123019.20000000001
Iteration 866, Best Score: 123019.20000000001
Iteration 867, Best Score: 123019.20000000001
Iteration 868, Best Score: 123019.20000000001
Iteration 869, Best Score: 123019.20000000001
Iteration 870, Best Score: 123019.20000000001
Iteration 871, Best Score: 123019.20000000001
Iteration 872, Best Score: 123019.20000000001
Iteration 873, Best Score: 123019.20000000001
Iteration 874, Best Score: 123019.20000000001
Iteration 875, Best Score: 123019.20000000001
Iteration 876, Best Score: 123019.20000000001
Iteration 877, Best Score: 123019.20000000001
Iteration 878, Best Score: 123019.20000000001
Iteration 879, Best Score: 123019.20000000001
Iteration 880, Best Score: 123019.

...

Iteration 4201, Best Score: 123019.20000000001
Iteration 4202, Best Score: 123019.20000000001
Iteration 4203, Best Score: 123019.20000000001
Iteration 4204, Best Score: 123019.20000000001
Iteration 4205, Best Score: 123019.20000000001
Iteration 4206, Best Score: 123019.20000000001
Iteration 4207, Best Score: 123019.20000000001
Iteration 4208, Best Score: 123019.20000000001
Iteration 4209, Best Score: 123019.20000000001
Iteration 4210, Best Score: 123019.20000000001
Iteration 4211, Best Score: 123019.20000000001
Iteration 4212, Best Score: 123019.20000000001
Iteration 4213, Best Score: 123019.20000000001
Iteration 4214, Best Score: 123019.20000000001
Iteration 4215, Best Score: 123019.20000000001
Iteration 4216, Best Score: 123019.20000000001
Iteration 4217, Best Score: 123019.20000000001
Iteration 4218, Best Score: 123019.20000000001
Iteration 4219, Best Score: 123019.20000000001
Iteration 4220, Best Score: 123019.20000000001
Iteration 4221, Best Score: 123019.20000000001
Iteration 422

Iteration 4378, Best Score: 123019.20000000001
Iteration 4379, Best Score: 123019.20000000001
Iteration 4380, Best Score: 123019.20000000001
Iteration 4381, Best Score: 123019.20000000001
Iteration 4382, Best Score: 123019.20000000001
Iteration 4383, Best Score: 123019.20000000001
Iteration 4384, Best Score: 123019.20000000001
Iteration 4385, Best Score: 123019.20000000001
Iteration 4386, Best Score: 123019.20000000001
Iteration 4387, Best Score: 123019.20000000001
Iteration 4388, Best Score: 123019.20000000001
Iteration 4389, Best Score: 123019.20000000001
Iteration 4390, Best Score: 123019.20000000001
Iteration 4391, Best Score: 123019.20000000001
Iteration 4392, Best Score: 123019.20000000001
Iteration 4393, Best Score: 123019.20000000001
Iteration 4394, Best Score: 123019.20000000001
Iteration 4395, Best Score: 123019.20000000001
Iteration 4396, Best Score: 123019.20000000001
Iteration 4397, Best Score: 123019.20000000001
Iteration 4398, Best Score: 123019.20000000001
Iteration 439

Iteration 4553, Best Score: 123019.20000000001
Iteration 4554, Best Score: 123019.20000000001
Iteration 4555, Best Score: 123019.20000000001
Iteration 4556, Best Score: 123019.20000000001
Iteration 4557, Best Score: 123019.20000000001
Iteration 4558, Best Score: 123019.20000000001
Iteration 4559, Best Score: 123019.20000000001
Iteration 4560, Best Score: 123019.20000000001
Iteration 4561, Best Score: 123019.20000000001
Iteration 4562, Best Score: 123019.20000000001
Iteration 4563, Best Score: 123019.20000000001
Iteration 4564, Best Score: 123019.20000000001
Iteration 4565, Best Score: 123019.20000000001
Iteration 4566, Best Score: 123019.20000000001
Iteration 4567, Best Score: 123019.20000000001
Iteration 4568, Best Score: 123019.20000000001
Iteration 4569, Best Score: 123019.20000000001
Iteration 4570, Best Score: 123019.20000000001
Iteration 4571, Best Score: 123019.20000000001
Iteration 4572, Best Score: 123019.20000000001
Iteration 4573, Best Score: 123019.20000000001
Iteration 457

Iteration 4730, Best Score: 123019.20000000001
Iteration 4731, Best Score: 123019.20000000001
Iteration 4732, Best Score: 123019.20000000001
Iteration 4733, Best Score: 123019.20000000001
Iteration 4734, Best Score: 123019.20000000001
Iteration 4735, Best Score: 123019.20000000001
Iteration 4736, Best Score: 123019.20000000001
Iteration 4737, Best Score: 123019.20000000001
Iteration 4738, Best Score: 123019.20000000001
Iteration 4739, Best Score: 123019.20000000001
Iteration 4740, Best Score: 123019.20000000001
Iteration 4741, Best Score: 123019.20000000001
Iteration 4742, Best Score: 123019.20000000001
Iteration 4743, Best Score: 123019.20000000001
Iteration 4744, Best Score: 123019.20000000001
Iteration 4745, Best Score: 123019.20000000001
Iteration 4746, Best Score: 123019.20000000001
Iteration 4747, Best Score: 123019.20000000001
Iteration 4748, Best Score: 123019.20000000001
Iteration 4749, Best Score: 123019.20000000001
Iteration 4750, Best Score: 123019.20000000001
Iteration 475

Iteration 4906, Best Score: 123019.20000000001
Iteration 4907, Best Score: 123019.20000000001
Iteration 4908, Best Score: 123019.20000000001
Iteration 4909, Best Score: 123019.20000000001
Iteration 4910, Best Score: 123019.20000000001
Iteration 4911, Best Score: 123019.20000000001
Iteration 4912, Best Score: 123019.20000000001
Iteration 4913, Best Score: 123019.20000000001
Iteration 4914, Best Score: 123019.20000000001
Iteration 4915, Best Score: 123019.20000000001
Iteration 4916, Best Score: 123019.20000000001
Iteration 4917, Best Score: 123019.20000000001
Iteration 4918, Best Score: 123019.20000000001
Iteration 4919, Best Score: 123019.20000000001
Iteration 4920, Best Score: 123019.20000000001
Iteration 4921, Best Score: 123019.20000000001
Iteration 4922, Best Score: 123019.20000000001
Iteration 4923, Best Score: 123019.20000000001
Iteration 4924, Best Score: 123019.20000000001
Iteration 4925, Best Score: 123019.20000000001
Iteration 4926, Best Score: 123019.20000000001
Iteration 492

In [18]:
score, routes = objective_function(solution, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 17, 18, 8, 2, 3, 14, 4, 11, 10, 15, 9, 0], [0, 19, 5, 12, 6, 16, 1, 13, 7, 20, 0]]
Objective function's values ---> 123019.20000000001


In [19]:
print("\nTabu Search:\n")
solution, routes = get_tabu_search_solution(1000, 100, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function_a_star, True)


Tabu Search:

Iteration 0, Best Score: 116110.1
Iteration 1, Best Score: 116110.1
Iteration 2, Best Score: 116110.1
Iteration 3, Best Score: 116110.1
Iteration 4, Best Score: 116110.1
Iteration 5, Best Score: 116110.1
Iteration 6, Best Score: 116110.1
Iteration 7, Best Score: 116110.1
Iteration 8, Best Score: 116110.1
Iteration 9, Best Score: 116110.1
Iteration 10, Best Score: 116110.1
Iteration 11, Best Score: 111074.6
Iteration 12, Best Score: 111074.6
Iteration 13, Best Score: 109637.4
Iteration 14, Best Score: 109637.4
Iteration 15, Best Score: 109637.4
Iteration 16, Best Score: 109215.4
Iteration 17, Best Score: 109215.4
Iteration 18, Best Score: 109215.4
Iteration 19, Best Score: 109215.4
Iteration 20, Best Score: 109215.4
Iteration 21, Best Score: 109215.4
Iteration 22, Best Score: 109215.4
Iteration 23, Best Score: 109215.4
Iteration 24, Best Score: 109215.4
Iteration 25, Best Score: 109215.4
Iteration 26, Best Score: 109215.4
Iteration 27, Best Score: 109215.4
Iteration 28, B

Iteration 230, Best Score: 106603.6
Iteration 231, Best Score: 106603.6
Iteration 232, Best Score: 106603.6
Iteration 233, Best Score: 106603.6
Iteration 234, Best Score: 106603.6
Iteration 235, Best Score: 106603.6
Iteration 236, Best Score: 106603.6
Iteration 237, Best Score: 106603.6
Iteration 238, Best Score: 106603.6
Iteration 239, Best Score: 106603.6
Iteration 240, Best Score: 106603.6
Iteration 241, Best Score: 106603.6
Iteration 242, Best Score: 106603.6
Iteration 243, Best Score: 106603.6
Iteration 244, Best Score: 106603.6
Iteration 245, Best Score: 106603.6
Iteration 246, Best Score: 106603.6
Iteration 247, Best Score: 106603.6
Iteration 248, Best Score: 106603.6
Iteration 249, Best Score: 106603.6
Iteration 250, Best Score: 106603.6
Iteration 251, Best Score: 106603.6
Iteration 252, Best Score: 106603.6
Iteration 253, Best Score: 106603.6
Iteration 254, Best Score: 106603.6
Iteration 255, Best Score: 106603.6
Iteration 256, Best Score: 106603.6
Iteration 257, Best Score: 1

Iteration 458, Best Score: 106603.6
Iteration 459, Best Score: 106603.6
Iteration 460, Best Score: 106603.6
Iteration 461, Best Score: 106603.6
Iteration 462, Best Score: 106603.6
Iteration 463, Best Score: 106603.6
Iteration 464, Best Score: 106603.6
Iteration 465, Best Score: 106603.6
Iteration 466, Best Score: 106603.6
Iteration 467, Best Score: 106603.6
Iteration 468, Best Score: 106603.6
Iteration 469, Best Score: 106603.6
Iteration 470, Best Score: 106603.6
Iteration 471, Best Score: 106603.6
Iteration 472, Best Score: 106603.6
Iteration 473, Best Score: 106603.6
Iteration 474, Best Score: 106603.6
Iteration 475, Best Score: 106603.6
Iteration 476, Best Score: 106603.6
Iteration 477, Best Score: 106603.6
Iteration 478, Best Score: 106603.6
Iteration 479, Best Score: 106603.6
Iteration 480, Best Score: 106603.6
Iteration 481, Best Score: 106603.6
Iteration 482, Best Score: 106603.6
Iteration 483, Best Score: 106603.6
Iteration 484, Best Score: 106603.6
Iteration 485, Best Score: 1

Iteration 686, Best Score: 106603.6
Iteration 687, Best Score: 106603.6
Iteration 688, Best Score: 106603.6
Iteration 689, Best Score: 106603.6
Iteration 690, Best Score: 106603.6
Iteration 691, Best Score: 106603.6
Iteration 692, Best Score: 106603.6
Iteration 693, Best Score: 106603.6
Iteration 694, Best Score: 106603.6
Iteration 695, Best Score: 106603.6
Iteration 696, Best Score: 106603.6
Iteration 697, Best Score: 106603.6
Iteration 698, Best Score: 106603.6
Iteration 699, Best Score: 106603.6
Iteration 700, Best Score: 106603.6
Iteration 701, Best Score: 106603.6
Iteration 702, Best Score: 106603.6
Iteration 703, Best Score: 106603.6
Iteration 704, Best Score: 106603.6
Iteration 705, Best Score: 106603.6
Iteration 706, Best Score: 106603.6
Iteration 707, Best Score: 106603.6
Iteration 708, Best Score: 106603.6
Iteration 709, Best Score: 106603.6
Iteration 710, Best Score: 106603.6
Iteration 711, Best Score: 106603.6
Iteration 712, Best Score: 106603.6
Iteration 713, Best Score: 1

Iteration 914, Best Score: 106603.6
Iteration 915, Best Score: 106603.6
Iteration 916, Best Score: 106603.6
Iteration 917, Best Score: 106603.6
Iteration 918, Best Score: 106603.6
Iteration 919, Best Score: 106603.6
Iteration 920, Best Score: 106603.6
Iteration 921, Best Score: 106603.6
Iteration 922, Best Score: 106603.6
Iteration 923, Best Score: 106603.6
Iteration 924, Best Score: 106603.6
Iteration 925, Best Score: 106603.6
Iteration 926, Best Score: 106603.6
Iteration 927, Best Score: 106603.6
Iteration 928, Best Score: 106603.6
Iteration 929, Best Score: 106603.6
Iteration 930, Best Score: 106603.6
Iteration 931, Best Score: 106603.6
Iteration 932, Best Score: 106603.6
Iteration 933, Best Score: 106603.6
Iteration 934, Best Score: 106603.6
Iteration 935, Best Score: 106603.6
Iteration 936, Best Score: 106603.6
Iteration 937, Best Score: 106603.6
Iteration 938, Best Score: 106603.6
Iteration 939, Best Score: 106603.6
Iteration 940, Best Score: 106603.6
Iteration 941, Best Score: 1

In [20]:
score, routes = objective_function_a_star(solution, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 5, 6, 16, 20, 7, 12, 18, 0], [0, 17, 8, 2, 14, 1, 3, 13, 4, 11, 15, 10, 9, 0]]
Objective function's values ---> 106603.6


# Hill-Climbing

In [21]:
def get_hc_solution(num_iterations, neighbor_operator, traveling_times, inspection_times, open_hours, objective_func, log=False):
    iteration = 0;
    best_solution = generate_random_solution() # Best solution after 'num_iterations' iterations without improvement
    best_score, best_routes = objective_func(best_solution, traveling_times, inspection_times, open_hours)
    
    print(f"Init Solution: score: {-best_score}")
    
    while iteration < num_iterations:
        iteration += 1
        neighbor_solution = neighbor_operator(best_solution)   #Test with Neighbour 1, 2 and 3
        neighbor_score, neighbor_routes = objective_func(neighbor_solution, traveling_times, inspection_times, open_hours)
        if neighbor_score > best_score:
            iteration = 0
            best_solution = neighbor_solution
            best_score = neighbor_score
            best_routes = neighbor_routes
            if log:
                (print(f"Solution {iteration}: score: {-best_score}"))
            
    print(f"Final Solution: solution: {best_solution}, score: {-best_score}, routes: {best_routes}")
    return best_solution, best_routes

In [22]:
print("Hill climbing:\n")
solution, routes = get_hc_solution(2000, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function, True)

Hill climbing:

Init Solution: score: 251219.90000000002
Solution 0: score: 238249.2
Solution 0: score: 191449.2
Solution 0: score: 180706.80000000002
Solution 0: score: 176079.1
Solution 0: score: 172673.1
Solution 0: score: 171163.4
Solution 0: score: 168709.19999999998
Solution 0: score: 158724.5
Solution 0: score: 158472.1
Solution 0: score: 156810.3
Final Solution: solution: [2 2 2 2 2 1 2 2 1 1 2 2 1 2 1 2 1 2 1 2], score: 156810.3, routes: [[0, 19, 17, 6, 13, 10, 15, 9, 0], [0, 5, 18, 12, 8, 16, 2, 1, 3, 14, 4, 11, 7, 20, 0]]


In [23]:
score, routes = objective_function(solution, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 17, 6, 13, 10, 15, 9, 0], [0, 5, 18, 12, 8, 16, 2, 1, 3, 14, 4, 11, 7, 20, 0]]
Objective function's values ---> 156810.3


In [24]:
print("Hill climbing:\n")
solution, routes = get_hc_solution(2000, get_neighbor_solution3, traveling_times, inspection_times, open_hours, objective_function_a_star, True)

Hill climbing:

Init Solution: score: 156910.6
Solution 0: score: 153991.30000000002
Solution 0: score: 143191.30000000002
Solution 0: score: 114142.40000000002
Solution 0: score: 113708.0
Solution 0: score: 113219.00000000003
Solution 0: score: 113184.5
Solution 0: score: 111981.20000000001
Solution 0: score: 111495.80000000002
Final Solution: solution: [1 2 1 2 1 1 1 2 2 1 2 2 2 2 1 2 2 2 1 1], score: 111495.80000000002, routes: [[0, 19, 5, 6, 1, 15, 10, 20, 7, 3, 0], [0, 17, 2, 14, 8, 16, 13, 4, 11, 12, 18, 9, 0]]


In [25]:
score, routes = objective_function_a_star(solution, traveling_times, inspection_times, open_hours)
print('Random state ---> {}'.format(routes))
print("Objective function's values ---> {}".format(-score))
map = display_routes(routes)
map

Random state ---> [[0, 19, 5, 6, 1, 15, 10, 20, 7, 3, 0], [0, 17, 2, 14, 8, 16, 13, 4, 11, 12, 18, 9, 0]]
Objective function's values ---> 111495.80000000002


# Problem 2

In [26]:
# convert the DataFrame to a dictionary
traveling_times = {}
for i in range(len(very_small_distances)):
    traveling_times[i] = very_small_distances.iloc[i].tolist()

num_establishments = len(traveling_times)-1
inspection_times = very_small_establishments[['Inspection Time']]
open_hours = very_small_establishments[['Opening Hours']]

def generate_random_solution():
    solution = np.zeros(num_establishments, dtype=int)
    values = list(range(1, vehicles + 1))
    np.random.shuffle(values)
    index = 0
    for i in range(num_establishments):
        solution[i] = values[index]
        index = (index + 1) % vehicles
    return solution


def objective_function2(chromosome, traveling_times, inspection_times, open_hours):
    total_travel_time = 0
    total_inspection_time = 0
    total_time_per_route = [0 for i in range(max(chromosome))]
    j = -1

    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]
    best_routes = []

    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)

    # Calculate the total travel time and inspection time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j += 1
        route_travel_time = 0
        route_inspection_time = 0
        current_time = 0
        # Add the departure/arrival establishment to the beginning and end of the route
        route = [0] + route + [0]
        
        # Use a greedy search algorithm to find the optimal path for the current route
        route = greedy_search(route, heuristic_distances)
        best_routes.append(route)
        
        start_time = ast.literal_eval(open_hours.iloc[route[1]][0]).index(1)  # Inspection start time of first establishment in route
        #print('start time: {}'.format(start_time))

        for i in range(1, len(route)):
            # Calculate traveling time between establishments
            current_establishment = route[i-1]
            next_establishment = route[i]
            travel_time = traveling_times[current_establishment][next_establishment]
            #route_travel_time += traveling_times[current_establishment][next_establishment]
            
            # Calculate inspection time and waiting time based on establishment's schedule
            current_inspection_time = inspection_times.iloc[next_establishment][0]*60 # Convert minutes to seconds
            current_open_hours = ast.literal_eval(open_hours.iloc[next_establishment][0])
            
            # Calculate waiting time if establishment is closed
            if i == 1:
                waiting_time = 0
            else:
                current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                #print('Current hour: {}'.format(current_hour))
                while current_open_hours[current_hour % 24] == 0:
                    current_hour += 1
                waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                #print('({} * 3600) - ({} + {} + ({}*3600))'.format(current_hour, current_time, travel_time, start_time))
                
            # Add inspection time and waiting time to current time
            if i > 1:
                current_time += max(waiting_time, 0)
            current_time += travel_time
            current_time += current_inspection_time
            
            if current_time > 28800:
                return -1000000000000, [0]
            
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(i, current_establishment, next_establishment, current_inspection_time, current_open_hours, travel_time, waiting_time, current_time))
            # Reset waiting time
            waiting_time = 0
            
            # Add inspection time and traveling time to route times
            #route_inspection_time += current_inspection_time
            route_travel_time = 0

        # Add total time for current route to total time per route
        total_time_per_route[j] = current_time
        #print('total time per route ----> {}'.format(total_time_per_route))

    # Add total time for all routes to total travel time
    total_travel_time += sum(total_time_per_route)
    
    return -total_travel_time, best_routes # Minimize the sum of inspection and travel time

def objective_function_a_star2(chromosome, traveling_times, inspection_times, open_hours):
    
    total_travel_time = 0
    l = 1
    
    # Initialize an empty list to store the routes
    routes = [[] for i in range(max(chromosome))]
    best_routes = []
    
    # Assign each establishment to a route
    for i in range(len(chromosome)):
        routes[chromosome[i]-1].append(i+1)
    
    # Calculate the total travel time for all routes
    for route in routes:
        if len(route) == 0:
            continue
        j = 1
        current_time = 0
        
        # Create a set with all establishments in the route
        route_establishments = set(route)
        route_len = len(route_establishments)
        
        # Add the depot to the beginning of the route
        route = [0]
        
        # Initialize the route travel time as zero
        route_travel_time = 0
        
        while route_establishments:
            # Calculate the A* algorithm for the next establishment
            current_establishment = route[-1]
            next_establishment = None
            best_score = float('inf')
            for e in route_establishments:
                i = 1
                # Calculate the total time (travel time, inspection time, waiting time, heuristic value) for the next establishment
                travel_time = traveling_times[route[-1]][e]
                inspection_time = inspection_times.iloc[e][0] * 60
                current_open_hours = ast.literal_eval(open_hours.iloc[e][0])
                if len(route_establishments) == route_len:
                    start_time = current_open_hours.index(1)
                i += 1
                # Calculate waiting time if establishment is closed
                if len(route) == 1:
                    waiting_time = 0
                else:
                    current_hour = int((current_time + travel_time + (start_time*3600))/3600)
                    while current_open_hours[current_hour % 24] == 0:
                        current_hour += 1
                    waiting_time = (current_hour * 3600) - (current_time + travel_time + (start_time*3600))
                
                # Add inspection time, waiting time, and travel time to the current time
                time_to_next = max(waiting_time, 0) + inspection_time + travel_time
                total_time = current_time + time_to_next
                heuristic_value = heuristic_distances[e][0]
                score = total_time + heuristic_value
                
                # If the current establishment has the smallest score, update next_establishment
                if score < best_score:
                    '''
                    best_waiting_time = max(waiting_time, 0)
                    best_open_hours = current_open_hours
                    best_inspection_time = inspection_time
                    if len(route) == 1:
                        best_start_time = start_time
                    else:
                        best_current_hour = current_hour
                    best_current_time = total_time
                    best_travel_time = travel_time
                    '''
                    time_to_next_establishment = time_to_next
                    next_establishment = e
                    best_score = score
            '''
            if len(route) > 1:
                print('Current hour: {}'.format(best_current_hour))
                print('({} * 3600) - ({} + {} + ({}*3600))'.format(best_current_hour, current_time, best_travel_time, best_start_time))
            else:
                print('start time: {}'.format(best_start_time))
            '''
            
            # Add next establishment to route and remove from set
            route_establishments.remove(next_establishment)
            route.append(next_establishment)
            
            #print('Update current time: {} + {}'.format(current_time, time_to_next_establishment))
            # Update the current time and route travel time
            current_time += time_to_next_establishment
            #route_travel_time += travel_time
            #print('Iteration {} ---> current establishment: {}, next establishment: {}, next inspection time: {}, open hours array: {}, travel time: {}, waiting time: {}, current time: {}'.format(j, current_establishment, next_establishment, best_inspection_time, best_open_hours, best_travel_time, best_waiting_time, best_current_time))
        
            if current_time > 28800:
                return -1000000000000, [0]
        
            j += 1

        route.append(0)
        best_routes.append(route)
        #print('Route: {}'.format(route))
        # Calculate the total time for the current route and add to total time per route
        total_time_per_route = current_time + traveling_times[route[-2]][0]       
        #print('Traveling time back to the depot: {}'.format(traveling_times[route[-2]][0]))
        #print('Total travel time in route {}: {}'.format(l, total_time_per_route))
        total_travel_time += total_time_per_route
        l += 1
        
    return -total_travel_time, best_routes # Minimize the total travel time

In [27]:
vehicles = 1

while True:
    print("\nSimulated Annealing:")
    print(f"Number of vehicles: {vehicles}\n")
    if vehicles == 1:
        best_solution, routes = get_sa_solution(1, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function2,True)
        best_score, best_routes = objective_function2(best_solution, traveling_times, inspection_times, open_hours)
    else:
        best_solution, routes = get_sa_solution(2000, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function2,True)
        best_score, best_routes = objective_function2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        routes = best_routes
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break
    vehicles += 1

map = display_routes(routes)
map


Simulated Annealing:
Number of vehicles: 1

Init Solution:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], score: 1000000000000
Final Solution: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], score: 1000000000000

Simulated Annealing:
Number of vehicles: 2

Init Solution:  [1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2], score: 1000000000000
Final Solution: [1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2], score: 1000000000000

Simulated Annealing:
Number of vehicles: 3

Init Solution:  [3 2 1 3 2 1 3 2 1 3 2 1 3 2 1 3 2 1 3 2], score: 1000000000000
Final Solution: [3 2 1 3 2 1 3 2 1 3 2 1 3 2 1 3 2 1 3 2], score: 1000000000000

Simulated Annealing:
Number of vehicles: 4

Init Solution:  [2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1], score: 1000000000000
Final Solution: [2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1], score: 1000000000000

Simulated Annealing:
Number of vehicles: 5

Init Solution:  [4 1 5 2 3 4 1 5 2 3 4 1 5 2 3 4 1 5 2 3], score: 1000000000000
Final Solution: [4 1 5 2 3 4 1 5 2 3 4 1 5 2 3 4 1 5 2 

C:\Users\luish\AppData\Local\Temp\ipykernel_14028\4165138825.py:21: RuntimeWarning: overflow encountered in exp
  if delta > 0 or np.exp(-delta/temperature) > random.random():


Solution: score: 162725.7,  Temp: 979.2086759647051
Solution: score: 162348.90000000002,  Temp: 975.2977125970466
Solution: score: 162160.7,  Temp: 964.6229185299474
Solution: score: 161838.7,  Temp: 963.6582956114174
Solution: score: 159365.0,  Temp: 957.8907814534662
Solution: score: 157888.8,  Temp: 955.0199818235594
Solution: score: 157694.4,  Temp: 953.1108968798942
Solution: score: 157558.69999999998,  Temp: 948.3548639781192
Solution: score: 151948.19999999998,  Temp: 943.6225637280606
Solution: score: 151175.9,  Temp: 934.2286880693634
Solution: score: 150969.5,  Temp: 919.3926150309798
Solution: score: 150969.49999999997,  Temp: 777.1467460721302
Solution: score: 148419.3,  Temp: 723.1330071735641
Solution: score: 147357.09999999998,  Temp: 664.1783033340388
Solution: score: 146857.8,  Temp: 626.7336333646191
Solution: score: 146374.0,  Temp: 602.147005002946
Final Solution: [5 9 6 8 7 7 1 3 4 4 8 3 2 9 4 5 1 6 7 2], score: 146374.0
Final Solution: [5 9 6 8 7 7 1 3 4 4 8 3 2 9

In [28]:
vehicles = 1

while True:
    print("\nSimulated Annealing:")
    print(f"Number of vehicles: {vehicles}\n")
    if vehicles == 1:
        best_solution = get_sa_solution(1, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function_a_star2,True)[0]
        best_score, best_routes = objective_function_a_star2(best_solution, traveling_times, inspection_times, open_hours)
    else:
        best_solution = get_sa_solution(2000, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function_a_star2,True)[0]
        best_score, best_routes = objective_function_a_star2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        routes = best_routes
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break
    vehicles += 1

map = display_routes(routes)
map


Simulated Annealing:
Number of vehicles: 1

Init Solution:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], score: 1000000000000
Final Solution: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], score: 1000000000000

Simulated Annealing:
Number of vehicles: 2

Init Solution:  [1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2], score: 1000000000000
Final Solution: [1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2], score: 1000000000000

Simulated Annealing:
Number of vehicles: 3

Init Solution:  [3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1], score: 1000000000000
Final Solution: [3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1], score: 1000000000000

Simulated Annealing:
Number of vehicles: 4

Init Solution:  [4 3 1 2 4 3 1 2 4 3 1 2 4 3 1 2 4 3 1 2], score: 1000000000000
Solution: score: 116108.09999999999,  Temp: 972.3747443770956


C:\Users\luish\AppData\Local\Temp\ipykernel_14028\4165138825.py:21: RuntimeWarning: overflow encountered in exp
  if delta > 0 or np.exp(-delta/temperature) > random.random():


Final Solution: [2 3 1 2 4 3 1 2 4 3 1 2 4 3 1 2 4 3 1 4], score: 116108.09999999999
Final Solution: [2 3 1 2 4 3 1 2 4 3 1 2 4 3 1 2 4 3 1 4], score: 116108.09999999999, num_vehicles: 4


In [29]:
vehicles = 1

while True:
    print("\nTabu Search:")
    print(f"Number of vehicles: {vehicles}\n")
    if vehicles == 1:
        best_solution = get_tabu_search_solution(1, 1, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function2, True)[0]
        best_score, best_routes = objective_function2(best_solution, traveling_times, inspection_times, open_hours)
    else:
        best_solution = get_tabu_search_solution(2000, 200, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function2, True)[0]
        best_score, best_routes = objective_function2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        routes = best_routes
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break
    vehicles += 1


Tabu Search:
Number of vehicles: 1

Iteration 0, Best Score: 1000000000000
Best solution: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], Best Score: 1000000000000, Routes: [0]

Tabu Search:
Number of vehicles: 2

Iteration 0, Best Score: 1000000000000
Iteration 1, Best Score: 1000000000000
Iteration 2, Best Score: 1000000000000
Iteration 3, Best Score: 1000000000000
Iteration 4, Best Score: 1000000000000
Iteration 5, Best Score: 1000000000000
Iteration 6, Best Score: 1000000000000
Iteration 7, Best Score: 1000000000000
Iteration 8, Best Score: 1000000000000
Iteration 9, Best Score: 1000000000000
Iteration 10, Best Score: 1000000000000
Iteration 11, Best Score: 1000000000000
Iteration 12, Best Score: 1000000000000
Iteration 13, Best Score: 1000000000000
Iteration 14, Best Score: 1000000000000
Iteration 15, Best Score: 1000000000000
Iteration 16, Best Score: 1000000000000
Iteration 17, Best Score: 1000000000000
Iteration 18, Best Score: 1000000000000
Iteration 19, Best Score: 1000000000000


Iteration 201, Best Score: 1000000000000
Iteration 202, Best Score: 1000000000000
Iteration 203, Best Score: 1000000000000
Iteration 204, Best Score: 1000000000000
Iteration 205, Best Score: 1000000000000
Iteration 206, Best Score: 1000000000000
Iteration 207, Best Score: 1000000000000
Iteration 208, Best Score: 1000000000000
Iteration 209, Best Score: 1000000000000
Iteration 210, Best Score: 1000000000000
Iteration 211, Best Score: 1000000000000
Iteration 212, Best Score: 1000000000000
Iteration 213, Best Score: 1000000000000
Iteration 214, Best Score: 1000000000000
Iteration 215, Best Score: 1000000000000
Iteration 216, Best Score: 1000000000000
Iteration 217, Best Score: 1000000000000
Iteration 218, Best Score: 1000000000000
Iteration 219, Best Score: 1000000000000
Iteration 220, Best Score: 1000000000000
Iteration 221, Best Score: 1000000000000
Iteration 222, Best Score: 1000000000000
Iteration 223, Best Score: 1000000000000
Iteration 224, Best Score: 1000000000000
Iteration 225, B

Iteration 402, Best Score: 1000000000000
Iteration 403, Best Score: 1000000000000
Iteration 404, Best Score: 1000000000000
Iteration 405, Best Score: 1000000000000
Iteration 406, Best Score: 1000000000000
Iteration 407, Best Score: 1000000000000
Iteration 408, Best Score: 1000000000000
Iteration 409, Best Score: 1000000000000
Iteration 410, Best Score: 1000000000000
Iteration 411, Best Score: 1000000000000
Iteration 412, Best Score: 1000000000000
Iteration 413, Best Score: 1000000000000
Iteration 414, Best Score: 1000000000000
Iteration 415, Best Score: 1000000000000
Iteration 416, Best Score: 1000000000000
Iteration 417, Best Score: 1000000000000
Iteration 418, Best Score: 1000000000000
Iteration 419, Best Score: 1000000000000
Iteration 420, Best Score: 1000000000000
Iteration 421, Best Score: 1000000000000
Iteration 422, Best Score: 1000000000000
Iteration 423, Best Score: 1000000000000
Iteration 424, Best Score: 1000000000000
Iteration 425, Best Score: 1000000000000
Iteration 426, B

Iteration 614, Best Score: 1000000000000
Iteration 615, Best Score: 1000000000000
Iteration 616, Best Score: 1000000000000
Iteration 617, Best Score: 1000000000000
Iteration 618, Best Score: 1000000000000
Iteration 619, Best Score: 1000000000000
Iteration 620, Best Score: 1000000000000
Iteration 621, Best Score: 1000000000000
Iteration 622, Best Score: 1000000000000
Iteration 623, Best Score: 1000000000000
Iteration 624, Best Score: 1000000000000
Iteration 625, Best Score: 1000000000000
Iteration 626, Best Score: 1000000000000
Iteration 627, Best Score: 1000000000000
Iteration 628, Best Score: 1000000000000
Iteration 629, Best Score: 1000000000000
Iteration 630, Best Score: 1000000000000
Iteration 631, Best Score: 1000000000000
Iteration 632, Best Score: 1000000000000
Iteration 633, Best Score: 1000000000000
Iteration 634, Best Score: 1000000000000
Iteration 635, Best Score: 1000000000000
Iteration 636, Best Score: 1000000000000
Iteration 637, Best Score: 1000000000000
Iteration 638, B

Iteration 814, Best Score: 1000000000000
Iteration 815, Best Score: 1000000000000
Iteration 816, Best Score: 1000000000000
Iteration 817, Best Score: 1000000000000
Iteration 818, Best Score: 1000000000000
Iteration 819, Best Score: 1000000000000
Iteration 820, Best Score: 1000000000000
Iteration 821, Best Score: 1000000000000
Iteration 822, Best Score: 1000000000000
Iteration 823, Best Score: 1000000000000
Iteration 824, Best Score: 1000000000000
Iteration 825, Best Score: 1000000000000
Iteration 826, Best Score: 1000000000000
Iteration 827, Best Score: 1000000000000
Iteration 828, Best Score: 1000000000000
Iteration 829, Best Score: 1000000000000
Iteration 830, Best Score: 1000000000000
Iteration 831, Best Score: 1000000000000
Iteration 832, Best Score: 1000000000000
Iteration 833, Best Score: 1000000000000
Iteration 834, Best Score: 1000000000000
Iteration 835, Best Score: 1000000000000
Iteration 836, Best Score: 1000000000000
Iteration 837, Best Score: 1000000000000
Iteration 838, B

...

Iteration 1028, Best Score: 136897.1
Iteration 1029, Best Score: 136897.1
Iteration 1030, Best Score: 136897.1
Iteration 1031, Best Score: 136897.1
Iteration 1032, Best Score: 136897.1
Iteration 1033, Best Score: 136897.1
Iteration 1034, Best Score: 136897.1
Iteration 1035, Best Score: 136897.1
Iteration 1036, Best Score: 136897.1
Iteration 1037, Best Score: 136897.1
Iteration 1038, Best Score: 136897.1
Iteration 1039, Best Score: 136897.1
Iteration 1040, Best Score: 136897.1
Iteration 1041, Best Score: 136897.1
Iteration 1042, Best Score: 136897.1
Iteration 1043, Best Score: 136897.1
Iteration 1044, Best Score: 136897.1
Iteration 1045, Best Score: 136897.1
Iteration 1046, Best Score: 136897.1
Iteration 1047, Best Score: 136897.1
Iteration 1048, Best Score: 136897.1
Iteration 1049, Best Score: 136897.1
Iteration 1050, Best Score: 136897.1
Iteration 1051, Best Score: 136897.1
Iteration 1052, Best Score: 136897.1
Iteration 1053, Best Score: 136897.1
Iteration 1054, Best Score: 136897.1
I

Iteration 1253, Best Score: 136897.1
Iteration 1254, Best Score: 136897.1
Iteration 1255, Best Score: 136897.1
Iteration 1256, Best Score: 136897.1
Iteration 1257, Best Score: 136897.1
Iteration 1258, Best Score: 136897.1
Iteration 1259, Best Score: 136897.1
Iteration 1260, Best Score: 136897.1
Iteration 1261, Best Score: 136897.1
Iteration 1262, Best Score: 136897.1
Iteration 1263, Best Score: 136897.1
Iteration 1264, Best Score: 136897.1
Iteration 1265, Best Score: 136897.1
Iteration 1266, Best Score: 136897.1
Iteration 1267, Best Score: 136897.1
Iteration 1268, Best Score: 136897.1
Iteration 1269, Best Score: 136897.1
Iteration 1270, Best Score: 136897.1
Iteration 1271, Best Score: 136897.1
Iteration 1272, Best Score: 136897.1
Iteration 1273, Best Score: 136897.1
Iteration 1274, Best Score: 136897.1
Iteration 1275, Best Score: 136897.1
Iteration 1276, Best Score: 136897.1
Iteration 1277, Best Score: 136897.1
Iteration 1278, Best Score: 136897.1
Iteration 1279, Best Score: 136897.1
I

Iteration 1475, Best Score: 136897.1
Iteration 1476, Best Score: 136897.1
Iteration 1477, Best Score: 136897.1
Iteration 1478, Best Score: 136897.1
Iteration 1479, Best Score: 136897.1
Iteration 1480, Best Score: 136897.1
Iteration 1481, Best Score: 136897.1
Iteration 1482, Best Score: 136897.1
Iteration 1483, Best Score: 136897.1
Iteration 1484, Best Score: 136897.1
Iteration 1485, Best Score: 136897.1
Iteration 1486, Best Score: 136897.1
Iteration 1487, Best Score: 136897.1
Iteration 1488, Best Score: 136897.1
Iteration 1489, Best Score: 136897.1
Iteration 1490, Best Score: 136897.1
Iteration 1491, Best Score: 136897.1
Iteration 1492, Best Score: 136897.1
Iteration 1493, Best Score: 136897.1
Iteration 1494, Best Score: 136897.1
Iteration 1495, Best Score: 136897.1
Iteration 1496, Best Score: 136897.1
Iteration 1497, Best Score: 136897.1
Iteration 1498, Best Score: 136897.1
Iteration 1499, Best Score: 136897.1
Iteration 1500, Best Score: 136897.1
Iteration 1501, Best Score: 136897.1
I

Iteration 1697, Best Score: 136897.1
Iteration 1698, Best Score: 136897.1
Iteration 1699, Best Score: 136897.1
Iteration 1700, Best Score: 136897.1
Iteration 1701, Best Score: 136897.1
Iteration 1702, Best Score: 136897.1
Iteration 1703, Best Score: 136897.1
Iteration 1704, Best Score: 136897.1
Iteration 1705, Best Score: 136897.1
Iteration 1706, Best Score: 136897.1
Iteration 1707, Best Score: 136897.1
Iteration 1708, Best Score: 136897.1
Iteration 1709, Best Score: 136897.1
Iteration 1710, Best Score: 136897.1
Iteration 1711, Best Score: 136897.1
Iteration 1712, Best Score: 136897.1
Iteration 1713, Best Score: 136897.1
Iteration 1714, Best Score: 136897.1
Iteration 1715, Best Score: 136897.1
Iteration 1716, Best Score: 136897.1
Iteration 1717, Best Score: 136897.1
Iteration 1718, Best Score: 136897.1
Iteration 1719, Best Score: 136897.1
Iteration 1720, Best Score: 136897.1
Iteration 1721, Best Score: 136897.1
Iteration 1722, Best Score: 136897.1
Iteration 1723, Best Score: 136897.1
I

Iteration 1922, Best Score: 136725.4
Iteration 1923, Best Score: 136725.4
Iteration 1924, Best Score: 136725.4
Iteration 1925, Best Score: 136725.4
Iteration 1926, Best Score: 136725.4
Iteration 1927, Best Score: 136725.4
Iteration 1928, Best Score: 136725.4
Iteration 1929, Best Score: 136725.4
Iteration 1930, Best Score: 136725.4
Iteration 1931, Best Score: 136725.4
Iteration 1932, Best Score: 136725.4
Iteration 1933, Best Score: 136725.4
Iteration 1934, Best Score: 136725.4
Iteration 1935, Best Score: 136725.4
Iteration 1936, Best Score: 136725.4
Iteration 1937, Best Score: 136725.4
Iteration 1938, Best Score: 136725.4
Iteration 1939, Best Score: 136725.4
Iteration 1940, Best Score: 136725.4
Iteration 1941, Best Score: 136725.4
Iteration 1942, Best Score: 136725.4
Iteration 1943, Best Score: 136725.4
Iteration 1944, Best Score: 136725.4
Iteration 1945, Best Score: 136725.4
Iteration 1946, Best Score: 136725.4
Iteration 1947, Best Score: 136725.4
Iteration 1948, Best Score: 136725.4
I

In [30]:
map = display_routes(routes)
map

In [31]:
vehicles = 1

while True:
    print("\nTabu Search:")
    print(f"Number of vehicles: {vehicles}\n")
    if vehicles == 1:
        best_solution = get_tabu_search_solution(1, 1, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function_a_star2, True)[0]
        best_score, best_routes = objective_function_a_star2(best_solution, traveling_times, inspection_times, open_hours)
    else:  
        best_solution = get_tabu_search_solution(1000, 100, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function_a_star2, True)[0]
        best_score, best_routes = objective_function_a_star2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        routes = best_routes
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break
    vehicles += 1


Tabu Search:
Number of vehicles: 1

Iteration 0, Best Score: 1000000000000
Best solution: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], Best Score: 1000000000000, Routes: [0]

Tabu Search:
Number of vehicles: 2

Iteration 0, Best Score: 1000000000000
Iteration 1, Best Score: 1000000000000
Iteration 2, Best Score: 1000000000000
Iteration 3, Best Score: 1000000000000
Iteration 4, Best Score: 1000000000000
Iteration 5, Best Score: 1000000000000
Iteration 6, Best Score: 1000000000000
Iteration 7, Best Score: 1000000000000
Iteration 8, Best Score: 1000000000000
Iteration 9, Best Score: 1000000000000
Iteration 10, Best Score: 1000000000000
Iteration 11, Best Score: 1000000000000
Iteration 12, Best Score: 1000000000000
Iteration 13, Best Score: 1000000000000
Iteration 14, Best Score: 1000000000000
Iteration 15, Best Score: 1000000000000
Iteration 16, Best Score: 1000000000000
Iteration 17, Best Score: 1000000000000
Iteration 18, Best Score: 1000000000000
Iteration 19, Best Score: 1000000000000


Iteration 198, Best Score: 1000000000000
Iteration 199, Best Score: 1000000000000
Iteration 200, Best Score: 1000000000000
Iteration 201, Best Score: 1000000000000
Iteration 202, Best Score: 1000000000000
Iteration 203, Best Score: 1000000000000
Iteration 204, Best Score: 1000000000000
Iteration 205, Best Score: 1000000000000
Iteration 206, Best Score: 1000000000000
Iteration 207, Best Score: 1000000000000
Iteration 208, Best Score: 1000000000000
Iteration 209, Best Score: 1000000000000
Iteration 210, Best Score: 1000000000000
Iteration 211, Best Score: 1000000000000
Iteration 212, Best Score: 1000000000000
Iteration 213, Best Score: 1000000000000
Iteration 214, Best Score: 1000000000000
Iteration 215, Best Score: 1000000000000
Iteration 216, Best Score: 1000000000000
Iteration 217, Best Score: 1000000000000
Iteration 218, Best Score: 1000000000000
Iteration 219, Best Score: 1000000000000
Iteration 220, Best Score: 1000000000000
Iteration 221, Best Score: 1000000000000
Iteration 222, B

Iteration 398, Best Score: 1000000000000
Iteration 399, Best Score: 1000000000000
Iteration 400, Best Score: 1000000000000
Iteration 401, Best Score: 1000000000000
Iteration 402, Best Score: 1000000000000
Iteration 403, Best Score: 1000000000000
Iteration 404, Best Score: 1000000000000
Iteration 405, Best Score: 1000000000000
Iteration 406, Best Score: 1000000000000
Iteration 407, Best Score: 1000000000000
Iteration 408, Best Score: 1000000000000
Iteration 409, Best Score: 1000000000000
Iteration 410, Best Score: 1000000000000
Iteration 411, Best Score: 1000000000000
Iteration 412, Best Score: 1000000000000
Iteration 413, Best Score: 1000000000000
Iteration 414, Best Score: 1000000000000
Iteration 415, Best Score: 1000000000000
Iteration 416, Best Score: 1000000000000
Iteration 417, Best Score: 1000000000000
Iteration 418, Best Score: 1000000000000
Iteration 419, Best Score: 1000000000000
Iteration 420, Best Score: 1000000000000
Iteration 421, Best Score: 1000000000000
Iteration 422, B

Iteration 598, Best Score: 1000000000000
Iteration 599, Best Score: 1000000000000
Iteration 600, Best Score: 1000000000000
Iteration 601, Best Score: 1000000000000
Iteration 602, Best Score: 1000000000000
Iteration 603, Best Score: 1000000000000
Iteration 604, Best Score: 1000000000000
Iteration 605, Best Score: 1000000000000
Iteration 606, Best Score: 1000000000000
Iteration 607, Best Score: 1000000000000
Iteration 608, Best Score: 1000000000000
Iteration 609, Best Score: 1000000000000
Iteration 610, Best Score: 1000000000000
Iteration 611, Best Score: 1000000000000
Iteration 612, Best Score: 1000000000000
Iteration 613, Best Score: 1000000000000
Iteration 614, Best Score: 1000000000000
Iteration 615, Best Score: 1000000000000
Iteration 616, Best Score: 1000000000000
Iteration 617, Best Score: 1000000000000
Iteration 618, Best Score: 1000000000000
Iteration 619, Best Score: 1000000000000
Iteration 620, Best Score: 1000000000000
Iteration 621, Best Score: 1000000000000
Iteration 622, B

Iteration 798, Best Score: 1000000000000
Iteration 799, Best Score: 1000000000000
Iteration 800, Best Score: 1000000000000
Iteration 801, Best Score: 1000000000000
Iteration 802, Best Score: 1000000000000
Iteration 803, Best Score: 1000000000000
Iteration 804, Best Score: 1000000000000
Iteration 805, Best Score: 1000000000000
Iteration 806, Best Score: 1000000000000
Iteration 807, Best Score: 1000000000000
Iteration 808, Best Score: 1000000000000
Iteration 809, Best Score: 1000000000000
Iteration 810, Best Score: 1000000000000
Iteration 811, Best Score: 1000000000000
Iteration 812, Best Score: 1000000000000
Iteration 813, Best Score: 1000000000000
Iteration 814, Best Score: 1000000000000
Iteration 815, Best Score: 1000000000000
Iteration 816, Best Score: 1000000000000
Iteration 817, Best Score: 1000000000000
Iteration 818, Best Score: 1000000000000
Iteration 819, Best Score: 1000000000000
Iteration 820, Best Score: 1000000000000
Iteration 821, Best Score: 1000000000000
Iteration 822, B

...

Iteration 191, Best Score: 113187.09999999999
Iteration 192, Best Score: 113187.09999999999
Iteration 193, Best Score: 113187.09999999999
Iteration 194, Best Score: 113187.09999999999
Iteration 195, Best Score: 113187.09999999999
Iteration 196, Best Score: 113187.09999999999
Iteration 197, Best Score: 113187.09999999999
Iteration 198, Best Score: 113187.09999999999
Iteration 199, Best Score: 113187.09999999999
Iteration 200, Best Score: 113187.09999999999
Iteration 201, Best Score: 113187.09999999999
Iteration 202, Best Score: 113187.09999999999
Iteration 203, Best Score: 113187.09999999999
Iteration 204, Best Score: 113187.09999999999
Iteration 205, Best Score: 113187.09999999999
Iteration 206, Best Score: 113187.09999999999
Iteration 207, Best Score: 113187.09999999999
Iteration 208, Best Score: 113187.09999999999
Iteration 209, Best Score: 113187.09999999999
Iteration 210, Best Score: 113187.09999999999
Iteration 211, Best Score: 113187.09999999999
Iteration 212, Best Score: 113187.

Iteration 370, Best Score: 113187.09999999999
Iteration 371, Best Score: 113187.09999999999
Iteration 372, Best Score: 113187.09999999999
Iteration 373, Best Score: 113187.09999999999
Iteration 374, Best Score: 113187.09999999999
Iteration 375, Best Score: 113187.09999999999
Iteration 376, Best Score: 113187.09999999999
Iteration 377, Best Score: 113187.09999999999
Iteration 378, Best Score: 113187.09999999999
Iteration 379, Best Score: 113187.09999999999
Iteration 380, Best Score: 113187.09999999999
Iteration 381, Best Score: 113187.09999999999
Iteration 382, Best Score: 113187.09999999999
Iteration 383, Best Score: 113187.09999999999
Iteration 384, Best Score: 113187.09999999999
Iteration 385, Best Score: 113187.09999999999
Iteration 386, Best Score: 113187.09999999999
Iteration 387, Best Score: 113187.09999999999
Iteration 388, Best Score: 113187.09999999999
Iteration 389, Best Score: 113187.09999999999
Iteration 390, Best Score: 113187.09999999999
Iteration 391, Best Score: 113187.

Iteration 549, Best Score: 113187.09999999999
Iteration 550, Best Score: 113187.09999999999
Iteration 551, Best Score: 113187.09999999999
Iteration 552, Best Score: 113187.09999999999
Iteration 553, Best Score: 113187.09999999999
Iteration 554, Best Score: 113187.09999999999
Iteration 555, Best Score: 113187.09999999999
Iteration 556, Best Score: 113187.09999999999
Iteration 557, Best Score: 113187.09999999999
Iteration 558, Best Score: 113187.09999999999
Iteration 559, Best Score: 113187.09999999999
Iteration 560, Best Score: 113187.09999999999
Iteration 561, Best Score: 113187.09999999999
Iteration 562, Best Score: 113187.09999999999
Iteration 563, Best Score: 113187.09999999999
Iteration 564, Best Score: 113187.09999999999
Iteration 565, Best Score: 113187.09999999999
Iteration 566, Best Score: 113187.09999999999
Iteration 567, Best Score: 113187.09999999999
Iteration 568, Best Score: 113187.09999999999
Iteration 569, Best Score: 113187.09999999999
Iteration 570, Best Score: 113187.

Iteration 728, Best Score: 113187.09999999999
Iteration 729, Best Score: 113187.09999999999
Iteration 730, Best Score: 113187.09999999999
Iteration 731, Best Score: 113187.09999999999
Iteration 732, Best Score: 113187.09999999999
Iteration 733, Best Score: 113187.09999999999
Iteration 734, Best Score: 113187.09999999999
Iteration 735, Best Score: 113187.09999999999
Iteration 736, Best Score: 113187.09999999999
Iteration 737, Best Score: 113187.09999999999
Iteration 738, Best Score: 113187.09999999999
Iteration 739, Best Score: 113187.09999999999
Iteration 740, Best Score: 113187.09999999999
Iteration 741, Best Score: 113187.09999999999
Iteration 742, Best Score: 113187.09999999999
Iteration 743, Best Score: 113187.09999999999
Iteration 744, Best Score: 113187.09999999999
Iteration 745, Best Score: 113187.09999999999
Iteration 746, Best Score: 113187.09999999999
Iteration 747, Best Score: 113187.09999999999
Iteration 748, Best Score: 113187.09999999999
Iteration 749, Best Score: 113187.

Iteration 907, Best Score: 111919.29999999999
Iteration 908, Best Score: 111919.29999999999
Iteration 909, Best Score: 111919.29999999999
Iteration 910, Best Score: 111919.29999999999
Iteration 911, Best Score: 111919.29999999999
Iteration 912, Best Score: 111919.29999999999
Iteration 913, Best Score: 111919.29999999999
Iteration 914, Best Score: 111919.29999999999
Iteration 915, Best Score: 111919.29999999999
Iteration 916, Best Score: 111919.29999999999
Iteration 917, Best Score: 111919.29999999999
Iteration 918, Best Score: 111919.29999999999
Iteration 919, Best Score: 111919.29999999999
Iteration 920, Best Score: 111919.29999999999
Iteration 921, Best Score: 111919.29999999999
Iteration 922, Best Score: 111919.29999999999
Iteration 923, Best Score: 111919.29999999999
Iteration 924, Best Score: 111919.29999999999
Iteration 925, Best Score: 111919.29999999999
Iteration 926, Best Score: 111919.29999999999
Iteration 927, Best Score: 111919.29999999999
Iteration 928, Best Score: 111919.

In [32]:
map = display_routes(routes)
map

In [33]:
vehicles= 1

while True:
    print("\nHill climbing:")
    print(f"Number of vehicles: {vehicles}\n")
    if vehicles == 1:
        best_solution = get_hc_solution(1, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function2, True)[0]
        best_score, best_routes = objective_function2(best_solution, traveling_times, inspection_times, open_hours)
    else:
        best_solution = get_hc_solution(1000, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function2, True)[0]
        best_score, best_routes = objective_function2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        routes = best_routes
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break
    vehicles += 1

map = display_routes(routes)
map


Hill climbing:
Number of vehicles: 1

Init Solution: score: 1000000000000
Final Solution: solution: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], score: 1000000000000, routes: [0]

Hill climbing:
Number of vehicles: 2

Init Solution: score: 1000000000000
Final Solution: solution: [1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2], score: 1000000000000, routes: [0]

Hill climbing:
Number of vehicles: 3

Init Solution: score: 1000000000000
Final Solution: solution: [2 1 3 2 1 3 2 1 3 2 1 3 2 1 3 2 1 3 2 1], score: 1000000000000, routes: [0]

Hill climbing:
Number of vehicles: 4

Init Solution: score: 1000000000000
Final Solution: solution: [3 1 2 4 3 1 2 4 3 1 2 4 3 1 2 4 3 1 2 4], score: 1000000000000, routes: [0]

Hill climbing:
Number of vehicles: 5

Init Solution: score: 1000000000000
Final Solution: solution: [1 4 5 2 3 1 4 5 2 3 1 4 5 2 3 1 4 5 2 3], score: 1000000000000, routes: [0]

Hill climbing:
Number of vehicles: 6

Init Solution: score: 1000000000000
Final Solution: solution: [5 1 4 6 

In [34]:
vehicles= 1

while True:
    print("\nHill climbing:")
    print(f"Number of vehicles: {vehicles}\n")
    if vehicles == 1:
        best_solution = get_hc_solution(1, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function_a_star2, True)[0]
        best_score, best_routes = objective_function_a_star2(best_solution, traveling_times, inspection_times, open_hours)
    else:
        best_solution = get_hc_solution(1000, get_neighbor_solution2, traveling_times, inspection_times, open_hours, objective_function_a_star2, True)[0]
        best_score, best_routes = objective_function_a_star2(best_solution, traveling_times, inspection_times, open_hours)
    if best_score > -10000000:
        routes = best_routes
        print(f"Final Solution: {best_solution}, score: {-best_score}, num_vehicles: {vehicles}")
        break
    vehicles += 1

map = display_routes(routes)
map


Hill climbing:
Number of vehicles: 1

Init Solution: score: 1000000000000
Final Solution: solution: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], score: 1000000000000, routes: [0]

Hill climbing:
Number of vehicles: 2

Init Solution: score: 1000000000000
Final Solution: solution: [1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2], score: 1000000000000, routes: [0]

Hill climbing:
Number of vehicles: 3

Init Solution: score: 1000000000000
Final Solution: solution: [3 2 1 3 2 1 3 2 1 3 2 1 3 2 1 3 2 1 3 2], score: 1000000000000, routes: [0]

Hill climbing:
Number of vehicles: 4

Init Solution: score: 1000000000000
Solution 0: score: 116108.1
Final Solution: solution: [2 3 4 2 1 3 4 2 1 3 4 2 1 3 4 2 1 3 4 1], score: 116108.1, routes: [[0, 17, 5, 13, 20, 9, 0], [0, 16, 1, 4, 12, 8, 0], [0, 6, 2, 14, 18, 10, 0], [0, 19, 15, 11, 3, 7, 0]]
Final Solution: [2 3 4 2 1 3 4 2 1 3 4 2 1 3 4 2 1 3 4 1], score: 116108.1, num_vehicles: 4
